contains all the code needed to prepare data

In [1]:
import requests
from bs4 import BeautifulSoup

import os
import json
import csv
import pandas as pd
import numpy as np

import time
from datetime import datetime

import bgg_data_func

# Games Master List from BGG

straightforward scraping

In [ ]:
session = requests.Session()
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)'\
           'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',
           'Accept':'text/html,application/xhtml+xml,application/xml;'\
           'q=0.9,image/webp,*/*;q=0.8'}
url = 'https://boardgamegeek.com/browse/boardgame?sort=rank'
req = session.get(url, headers=headers)

bs = BeautifulSoup(req.text, 'html.parser')

In [ ]:
ranking_table = bs.find('table', {'class': 'collection_table'})

In [ ]:
rows = ranking_table.findChildren(['th', 'tr'])

In [ ]:
bg_name_list = []
bg_link_list = []
bg_year_list = []

for row in rows[8:]:
    cell = row.findChildren(['td'])[2]
    bg_name = cell.find('a').text
    bg_link = cell.find('a')['href']
    bg_year = cell.find('span', {'class': 'smallerfont dull'}).text[1:-1]
    
    bg_name_list.append(bg_name)
    bg_link_list.append(bg_link)
    bg_year_list.append(bg_year)
    
games_master_list = pd.DataFrame(columns=['name', 'link', 'year'])
games_master_list['name'] = bg_name_list
games_master_list['link'] = bg_link_list
games_master_list['year'] = bg_year_list

In [ ]:
games_master_list.to_csv('games_master_list.csv', index=False)

we have games_master_list.csv, with top 100 games, year, and link (which contains ID too)


prob wont have to run this often

# Import Master List

In [2]:
df = pd.read_csv('games_master_list.csv')

game_ids = df['link'].map(lambda x: bgg_data_func.link_to_gameid(x))

# BGG API Calls

in this section, looping through the 100 games


for each game, looping through the ratings


save ratings by 50 (100 doesnt work for some reason) in a json file in json_data folder

In [3]:
game_ids[100]

'123260'

In [4]:
# cell saves all available pages for a given gamelist, change game_ids parameters
# works with other string IDs as well but must have a folder

for game_id in game_ids[85:100]:
    
    print(game_id)
    
    page_num = 1
    page_empty = False

    while not page_empty:

        page_empty = bgg_data_func.handle_one_api_request(game_id, page_num)

        seconds_to_sleep = 3 + np.random.uniform(0,1)
        time.sleep(seconds_to_sleep)
        
        page_num += 1
        
        if page_num % 50 == 0:
            print(page_num)
        
    time.sleep(10)

244521
50
100
150
200
250
198928
50
100
127023
50
100
150
200
250
300
285774
50
103885
50
100
150
200
250
300
350
400
196340
50
100
150
172287
50
100
150
200
182874
50
100
150
200
203993
50
100
150
175155
50
100
34635
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
150376
50
100
150
200
250
300
350
400
450
500
550
600
650
700
161970
50
100
150
200
250
300
104162
50
100
150
200
250
300
350
400
147020
50
100
150
200
250
300
350
400
450
500
550
600
650


In [8]:
# this one just gets info for one game, from a certain page num

game_id = '157354'
page_num = 359
rated_only = True

page_empty = False

while not page_empty:

    # get API response
    page_empty = bgg_data_func.handle_one_api_request(game_id, page_num, rated_only)

    seconds_to_sleep = 2 + np.random.uniform(0,1)
    time.sleep(seconds_to_sleep)

    page_num += 1
    
    if page_num % 50 == 0:
        print(page_num)

400
450
500
550


In [4]:
# and finally, this one is a given page num for a given game

game_id = '28143'
page_num = 808

bgg_data_func.handle_one_api_request(game_id, page_num, False)

False

# JSON Data Converter

this section takes the raw json files the API returns, converts them into usable csvs

In [3]:
# game_ids_to_convert = pd.concat([game_ids[49:100],(game_ids[-1:])])
game_ids_to_convert = game_ids[49:100]


In [4]:
game_ids_to_convert

49     28143
50    122515
51    199792
52     62219
53    159675
54     18602
55    125153
56    110327
57    157354
58    201808
59     12493
60        93
61    205896
62    146021
63    172386
64    229853
65     40834
66    161533
67    247763
68       521
69    192135
70     37111
71     73439
72    163412
73    178900
74    191189
75    200680
76    236457
77    144733
78    132531
79        42
80      2511
81    102680
82    170042
83     36218
84     30549
85    244521
86    198928
87    127023
88    285774
89    103885
90    196340
91    172287
92    182874
93    203993
94    175155
95     34635
96    150376
97    161970
98    104162
99    147020
Name: link, dtype: object

In [5]:
for game_id in game_ids_to_convert:
    print(game_id)
    bgg_data_func.create_csv_summary(game_id)

28143
122515
199792
62219
159675
18602
125153
110327
157354
201808
12493
93
205896
146021
172386
229853
40834
161533
247763
521
192135
37111
73439
163412
178900
191189
200680
236457
144733
132531
42
2511
102680
170042
36218
30549
244521
198928
127023
285774
103885
196340
172287
182874
203993
175155
34635
150376
161970
104162
147020


# Create Data Summary

this section loops through the csv summary files per game and creates files that can be imported to different models. saves them in data_inputs folder

In [6]:
summary_filename = "games_100"

In [7]:
game_ids_to_merge = game_ids[:100]

In [8]:
len(game_ids_to_merge)

100

In [10]:
for game_id in game_ids_to_merge:
    
    current_file = pd.read_csv('./json_data/game_' + game_id + '/' + game_id + '_summary.csv', header = None)
    
    current_file[2] = game_id
    
    with open('./data_input/' + summary_filename + '_summary.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerows(zip(current_file[0], current_file[2], current_file[1]))

In [13]:
# check length

ratings_file = open('./data_input/' + summary_filename + '_summary.csv')
reader = csv.reader(ratings_file)
lines= len(list(reader))
print(lines)

2369886
